# Import

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime

# Hàm gọi API

In [ ]:
def make_api_request(key,location,start_day):
    lon = location[0]
    lat = location[1]
    url = f'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={start_day}&appid={key}'
    r = requests.get(url)
    return r.json()

# Dataframe các thành phố sẽ lấy thông tin
Bởi vì mỗi lần gọi API của openweathermap chỉ cho lấy dữ liệu theo giờ của 5 ngày trước đó nên ta thực hiện lấy nhiều thành phố thuộc khu vực nam bộ vì các thành phố gần nhau có thể có khí hậu giống nhau

In [3]:
coords = [[106.650002,106.666672,106.099998,106.416672,105.76667,105.083328,106.349998,105.416672,104.48333,106.466667,
           105.116669,105.783333,106.966667],
          [10.96667,10.75,11.3,10.53333,10.3,10.01667,10.35,10.38333,10.38333,10.96667,10.7,10.03333,10.41667]]
city_names = ["THU DAU MOT","THANH PHO HO CHI MINH","TAY NINH","TAN AN","SA DEC","RACH GIA","MY THO","LONG XUYEN",
                 "HA TIEN","CU CHI","CHAU DOC","CAN THO","CAN GIO"]
cities_df = pd.DataFrame(data = coords, columns = city_names, index = ["long","lat"])
cities_df.head(5)

,THU DAU MOT,THANH PHO HO CHI MINH,TAY NINH,TAN AN,SA DEC,RACH GIA,MY THO,LONG XUYEN,HA TIEN,CU CHI,CHAU DOC,CAN THO,CAN GIO
long,106.650002,106.666672,106.099998,106.416672,105.76667,105.083328,106.349998,105.416672,104.48333,106.466667,105.116669,105.783333,106.966667
lat,10.966670,10.750000,11.300000,10.533330,10.30000,10.016670,10.350000,10.383330,10.38333,10.966670,10.700000,10.033330,10.416670


# Thực hiện lấy dữ liệu

In [4]:
key = 'db898287c9d4a7e5274c7fbf618e0425'
day = [1610150400,1610236800,1610323200,1610409600,1610496000]

In [9]:
def get_all_city_info(cities_df,key,day):
    columns = ['time','city_name','temp','pressure','humidity','clouds','wind_speed','weather description']
    raw_df = pd.DataFrame(columns=columns)
    for col in cities_df.columns:
        for each_day in day:
            location = [cities_df[col][0],cities_df[col][1]]
            res = make_api_request(key,location,each_day)
            for h in range(0,len(res['hourly'])):
                this_hour = res['hourly'][h]
                new_row = [datetime.utcfromtimestamp(this_hour['dt']).strftime('%Y-%m-%d %H:%M:%S'),col,
                           this_hour['temp'],this_hour['pressure'],this_hour['humidity'],this_hour['clouds'],
                           this_hour['wind_speed'],this_hour['weather'][0]['description']]
                new_row_sr = pd.Series(new_row,index=columns)
                raw_df = raw_df.append(new_row_sr,ignore_index = True)        
    return raw_df
 

In [10]:
raw_weather_df = get_all_city_info(cities_df,key,day)
raw_weather_df

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather description
0,2021-01-09 00:00:00,THU DAU MOT,297.15,1011,69,20,1.54,few clouds
1,2021-01-09 01:00:00,THU DAU MOT,298.15,1012,65,96,1.54,overcast clouds
2,2021-01-09 02:00:00,THU DAU MOT,299.15,1012,65,20,1.03,few clouds
3,2021-01-09 03:00:00,THU DAU MOT,301.15,1012,57,20,2.57,few clouds
4,2021-01-09 04:00:00,THU DAU MOT,302.15,1011,51,20,1.54,few clouds
...,...,...,...,...,...,...,...,...
1555,2021-01-13 19:00:00,CAN GIO,294.15,1010,83,20,1.54,few clouds
1556,2021-01-13 20:00:00,CAN GIO,293.15,1010,88,20,1.03,few clouds
1557,2021-01-13 21:00:00,CAN GIO,293.15,1010,88,40,0.51,scattered clouds
1558,2021-01-13 22:00:00,CAN GIO,292.15,1010,94,40,0.51,scattered clouds


# Xuất ra file csv

In [11]:
raw_weather_df.to_csv('raw_weather_df.csv')